In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv('/content/courses_new.csv')
college_df = pd.read_csv('/content/Book1.csv')

In [ ]:
X = df[['English', 'Maths', 'Physics','Biology','Chemistry','Accounting','Economics','Business Studies','History','Sociology',
        'Politics', 'High School CGPA', 'Stream_Point']]
y = df['Recommended Course']


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


xgb_classifier = xgb.XGBClassifier(random_state=42)
xgb_classifier.fit(X_train, y_train)

y_pred_xgb = xgb_classifier.predict(X_test)


knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

y_pred_knn = knn_classifier.predict(X_test)

svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train, y_train)

y_pred_svm = svm_classifier.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

ensemble_predictions = []
for i in range(len(y_test)):
    predictions = [y_pred_xgb[i], y_pred_knn[i], y_pred_svm[i]]
    majority_vote = Counter(predictions).most_common(1)[0][0]
    ensemble_predictions.append(majority_vote)


print(f"Accuracy of XGBoost classifier: {accuracy_xgb:.2f}")
print(f"Accuracy of K-Nearest Neighbors classifier: {accuracy_knn:.2f}")
print(f"Accuracy of Support Vector Machine classifier: {accuracy_svm:.2f}")
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Accuracy of the Ensemble model: {accuracy:.2f}")


import tabulate

def predict_and_recommend(input_data):

    new_data_predictions_xgb = xgb_classifier.predict(input_data)
    new_data_predictions_knn = knn_classifier.predict(input_data)
    new_data_predictions_svm = svm_classifier.predict(input_data)

    predictions = [new_data_predictions_xgb[0], new_data_predictions_knn[0], new_data_predictions_svm[0]]
    top_three = [x[0] for x in Counter(predictions).most_common(3)]


    predicted_courses = label_encoder.inverse_transform(top_three)

    recommended_courses = list(predicted_courses)


    print("Recommended Courses:")
    for course in recommended_courses:
        print(course)

    recommended_colleges = []

    for predicted_course in recommended_courses:
        colleges_for_course = college_df[college_df['Recommended Course'] == predicted_course]
        recommended_colleges.append(colleges_for_course)

    sorted_recommended_colleges = [colleges.sort_values(by='NIRF_Ranking_2023') for colleges in recommended_colleges]

    for i, course in enumerate(recommended_courses):
        print(f"\nTop 5 Colleges for {course}:")
        college_table = sorted_recommended_colleges[i][['College', 'NIRF_Ranking_2023']].head(5)

        print(college_table)

new_data = [[100, 100, 0, 0, 0,100,200,140,0,0,0,7,6]]
predict_and_recommend(new_data)




#joblib.dump(predict_and_recommend, 'course_recommendation_modell.pkl')


Accuracy of XGBoost classifier: 0.96
Accuracy of K-Nearest Neighbors classifier: 0.72
Accuracy of Support Vector Machine classifier: 0.23
Accuracy of the Ensemble model: 0.91
Recommended Courses:
Human Resource Management (B.Com)
International Business (B.Com)
Accounting (B.Com)

Top 5 Colleges for Human Resource Management (B.Com):
                                              College  NIRF_Ranking_2023
90               Shri Ram College of Commerce (Delhi)                  3
91            Lady Shri Ram College for Women (Delhi)                  6
92                           Loyola College (Chennai)                 15
93  St. Xavier's College (Autonomous), Mumbai (Mum...                 16
94                      Christ University (Bangalore)                 19

Top 5 Colleges for International Business (B.Com):
                                              College  NIRF_Ranking_2023
65          Indian Institute of Foreign Trade (Delhi)                  1
66                      Chris

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
# Save the XGBoost classifier
joblib.dump(xgb_classifier, 'xgb_classifier_model.pkl')

# Save the K-Nearest Neighbors classifier
joblib.dump(knn_classifier, 'knn_classifier_model.pkl')

# Save the Support Vector Machine classifier
joblib.dump(svm_classifier, 'svm_classifier_model.pkl')

['svm_classifier_model.pkl']